# Imports and definitions

In [1]:
import mlflow
import seaborn as sns
import sys
import pandas as pd
import numpy as np

sys.path.append("../") # go to parent dir
mlflow.tracking.set_tracking_uri('../mlruns')

labels_methods = {
    'cam': 'CAM', 'gradcam': 'Grad-CAM', 'gradcam++': 'Grad-CAM++', 'scorecam': 'Score-CAM', 'minmaxcam': 'MinMaxCAM'}

# Functions

In [2]:
def get_class_labels(split='val'):
    """
    image_ids.txt has the structure

    <path>,<integer_class_label>
    path/to/image1.jpg,0
    path/to/image2.jpg,1
    path/to/image3.jpg,1
    ...
    """
    class_labels = {}
    path = f'/Users/goemaereg/github.com/thesis/data/metadata/ILSVRC/{split}/class_labels.txt'
    with open(path) as f:
        for line in f.readlines():
            image_id, class_label_string = line.strip('\n').split(',')
            class_labels[image_id] = int(class_label_string)
    return class_labels

def get_label_names():
    label_names = {}
    with open('/Users/goemaereg/github.com/thesis/other/imagenet1000/imagenet1000_clsidx_to_labels.txt', 'r') as fp:
        for line in fp.readlines():
            label, name = line[:-1].split(':')
            label = label.strip()
            name = name.strip()[1:-2]
            # remove alternative names
            name = name.split(',')[0]
            label_names[label] = name
    return label_names

In [3]:
class_labels = get_class_labels()
label_names = get_label_names()

In [4]:
from mlflow import MlflowClient

def mlflow_create_filter(conditions=None):
    if conditions is None:
        return ''
    keys = conditions.keys()
    values = [ f'"{v}"' if type(v) == str else v for v in conditions.values()]
    filter = ' AND '.join([f'{k} = {v}' for k,v in zip(keys, values)])
    return filter

def mlflow_get_runs(experiment_name, conditions=None):
    client = MlflowClient()
    exp_ids = [client.get_experiment_by_name(experiment_name).experiment_id]
    filter = mlflow_create_filter(conditions)
    runs = mlflow.search_runs(experiment_ids=exp_ids, filter_string=filter)
    return runs

def get_runs(experiment_name, filter=''):
    client = MlflowClient()
    exp_ids = [client.get_experiment_by_name(experiment_name).experiment_id]
    runs = mlflow.search_runs(experiment_ids=exp_ids, filter_string=filter)
    return runs

def get_best_run(runs, best_metrics):
    if runs is None or runs.shape[0] == 0:
        return None
    for metric in best_metrics:
        runs = runs.loc[runs[f'metrics.{metric}'] >= runs[f'metrics.{metric}'].max()]
    # pick first in the list
    run = runs.loc[runs.index[0]]
    return run

def epoch_values(x, metric):
    run_id = x.run_id
    client = MlflowClient()
    values = [[m.value for m in client.get_metric_history(id, metric)] for id in run_id]
    steps = [list(range(len(v))) for v in values]
    return steps, values

def epoch_steps(x):
    steps, _ = epoch_values(x, 'val_accuracy')
    return steps

def epoch_val_accuracy(x):
    _, values = epoch_values(x, 'val_accuracy')
    return values

def epoch_maxboxaccv3(x):
    _, values = epoch_values(x, 'val_MaxBoxAccV3')
    return values

In [5]:
def dataframe_to_latex(runs, split='test', metrics=None, label='', caption=''):
    cols_metrics = [f'metrics.{split}_{metric}' for metric in metrics]
    cols = ['tags.method', 'tags.architecture', 'tags.architecture_type', 'tags.dataset', 'tags.dataset_spec'] + cols_metrics
    cols_renamed = [c.split('.')[-1] for c in cols]
    mapper = dict(zip(cols, cols_renamed))
    df = runs.loc[:, cols].rename(columns=mapper)
    mapper = dict([(f'{split}_{metric}', metric) for metric in metrics])
    df = df.loc[:, df.columns].rename(columns=mapper)
    # df.loc[:, 'dataset'] = df['dataset'].str.cat(df['dataset_spec'], sep='\_').str.lower()
    # pd.set_option("display.precision", 2)
    for metric in metrics:
        df[metric] *= 100 # to percentage
    df = df.replace({'method': labels_methods})
    if dataset == 'SYNTHETIC':
        specs = df['dataset_spec'].unique()
        cols_pivot = ['dataset_spec']
        df = df.pivot(index='method', columns=cols_pivot, values=metrics)
        cols = metrics
        df = df.loc[:, cols]
        df = df.sort_index()
        latex_table = df.to_latex(float_format="%.2f", multicolumn=True,
                              label=label, caption=caption, position="ht")

        cols_format = dict([((m, spec), '{:.2f}') for m in metrics for spec in specs])
        s = df.style
        s = s.format(cols_format)
        # s = s.highlight_max(subset=metrics, axis=0, props='bfseries: ;')
        s = s.highlight_max(subset=metrics, axis=0, props='color:{teal}; bfseries: ;')
        s = s.highlight_min(subset=metrics, axis=0, props='color:{purple}; bfseries: ;')

        latex_table2 = s.to_latex(
                column_format="lrrrrrrrr", position="ht", position_float="centering",
                hrules=False, label=label, caption=caption,
                multirow_align="t", multicol_align="c")
    else:
        cols = ['method'] + metrics
        df = df.loc[:, cols]
        # remove minmaxcam from dataset for Imagenet VGG16-GAP (could not be trained enough)
        if 'tags.architecture' in conditions and conditions['tags.architecture'] == 'vgg16':
            if 'tags.architecture_type' in conditions and conditions['tags.architecture_type'] == 'cam':
                df = df.loc[df['method'] != 'MinMaxCAM']
        df = df.sort_values('method')
        latex_table = df.to_latex(float_format="%.2f", multicolumn=True,
                      label=label, caption=caption, position="ht", index=False)

        cols_format = dict([(m, '{:.2f}') for m in metrics])
        s = df.set_index('method').style
        s = s.format(cols_format)
        s = s.highlight_max(subset=metrics, axis=0, props='color:{teal}; bfseries: ;')
        s = s.highlight_min(subset=metrics, axis=0, props='color:{purple}; bfseries: ;')

        latex_table2 = s.to_latex(
                column_format="lrrr", position="ht", position_float="centering",
                hrules=False, label=label, caption=caption,
                multirow_align="t", multicol_align="c")
    print(latex_table)
    print(latex_table2)

def mlflow_runs_to_latex(experiment_name, conditions=None, split='test', metrics=None, label='', caption=''):
    if conditions is not None and 'tags.dataset' in conditions and conditions['tags.dataset'] == 'SYNTHETIC':
        dataset = 'SYNTHETIC'
    else:
        dataset = 'ILSVRC'
    if metrics is None:
        metrics = ['MaxBoxAcc', 'MaxBoxAccV2', 'MaxBoxAccV3_precision', 'MaxBoxAccV3_recall']
        if dataset == 'SYNTHETIC':
            metrics.append('PxAP')
    runs = mlflow_get_runs(experiment_name, conditions=conditions)
    if experiment_name == 'mwsol_resnet50_imagenet':
        runs = runs.loc[runs.start_time > '2023-05-01']
    dataframe_to_latex(runs, split=split, metrics=metrics, label=label, caption=caption)

In [6]:
import matplotlib.pyplot as plt
import os, json
import numpy as np

def get_mlflow_boxacc_prcurve(experiment_name = 'wsol2', conditions=None, split='test', box_metric='MaxBoxAccV3', mask_metric=None):
    boxacc = {}
    prcurve = {}
    iou_thresholds = [30, 50, 70]
    runs = mlflow_get_runs(experiment_name, conditions)
    for row in runs.itertuples(index=False):
        artifact_uri = row.artifact_uri
        artifact_path = artifact_uri[len('file://'):]
        config_path = os.path.join(artifact_path, 'state/config.json')
        with open(config_path) as fp:
            _config = json.load(fp)
        boxacc[_config['wsol_method']] = {}
        for iou_threshold in iou_thresholds:
            boxacc_path = os.path.join(artifact_path, f'data/{split}/{box_metric}_box_acc_iou_{iou_threshold}.json')
            with open(boxacc_path) as fp:
                _boxacc = json.load(fp)
                values = {k: _boxacc[k] for k in ['cam_threshold', 'box_accuracy']}
                boxacc[_config['wsol_method']][iou_threshold] = values
        if mask_metric is not None:
            prcurve_path = os.path.join(artifact_path, f'data/{split}/pr_curve.json')
            with open(prcurve_path) as fp:
                # _prcurve = json.load(fp)
                # values = {k: _prcurve[k] for k in ['precision', 'recall']}
                prcurve[_config['wsol_method']] = json.load(fp)
        else:
            prcurve = None
    return boxacc, prcurve

def plot_boxacc(boxacc, title, iou_thresholds=None):
    # plot lines
    if iou_thresholds is None:
        iou_thresholds = [30, 50, 70]
    for method, iou_values in boxacc.items():
        for iou_threshold, values in iou_values.items():
            if iou_threshold not in iou_thresholds:
                continue
            x = np.asarray(values['cam_threshold'])
            y = np.asarray(values['box_accuracy'])
            y *= 100 # convert to percentage
            xmax = x[np.argmax(y)]
            ymax = y.max()
            text = f"({xmax:.2f}, {ymax:.2f})"
            lb_method = labels_methods[method]
            label = lb_method if len(iou_thresholds) == 1 else f'{lb_method} {iou_threshold}'
            label = f'{label} {text}'
            plt.plot(x, y, label=label)
            plt.plot(xmax, ymax, 'ro')
            # plt.text(xmax + 0.01, ymax + 0.01, text)
    plt.xlabel("Score map threshold")
    plt.ylabel("BoxAcc")
    plt.xlim((-0.05, 1.05))
    plt.ylim((-5, 105))
    plt.title(title)
    plt.legend()
    plt.show()

def plot_prcurve(curve, title):
    # plot lines
    for method, values in curve.items():
        label = f"{method} (auc {values['auc']:.2f})"
        plt.plot(values['recall'], values['precision'], label=label)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.xlim((-0.05, 1.05))
    plt.ylim((-0.05, 1.05))
    plt.title(title)
    plt.legend()
    plt.show()

# Localization synthetic datasets

## VGG16-GAP

### Classification versus localization

In [ ]:
metric = 'MaxBoxAccV3'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, col_wrap=4, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_vgg16_gap_minmaxcam_synthetic.png')

In [ ]:
metric = 'MaxBoxAccV3'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'minmaxcam',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, col_wrap=4, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_vgg16_gap_minmaxcam_synthetic.png')

### MaxBoxAccV3 recall

#### non-regularized

In [ ]:
metric = 'MaxBoxAccV3_recall'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
runs = mlflow_get_runs(experiment_name, conditions)
dataframe_to_latex(runs=runs, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'MaxBoxAccV3_recall'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
runs = mlflow_get_runs(experiment_name, conditions)
dataframe_to_latex(runs=runs, split=split, metrics=[metric], caption=caption, label=label)

### MaxBoxAccV3 precision

#### non-regularized

In [ ]:
metric = 'MaxBoxAccV3_precision'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'MaxBoxAccV3_precision'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### PxAP

#### non-regularized

In [ ]:
metric = 'PxAP'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
# conditions |= {
#     'tags.dataset_spec': 'd4b'
# }
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'PxAP'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
# conditions |= {
#     'tags.dataset_spec': 'd4b'
# }
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### Visual explanation examples

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image

split = 'test'
image_id = 10
iter = 0
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol2_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')

speclist = sorted(df['tags.dataset_spec'].unique())
for spec in speclist:
    df_spec = df.loc[df['tags.dataset_spec'] == spec]
    df_spec = df_spec.loc[:, ['tags.method', 'tags.dataset_spec', 'artifact_uri']]
    df_spec = df_spec.sort_values('tags.method')
    spec_dict = df_spec.set_index('tags.method').to_dict()
    uris = spec_dict['artifact_uri']
    spec = '_'.join(list(spec))
    filebase = f'SYNTHETIC_{spec}_test_{image_id}'
    file_img = filebase + f'_img_{iter}.png'
    file_ann = filebase + f'_ann_{iter}.png'
    file_cam = filebase + f'_cam_{iter}.png'
    file_seg = filebase + f'_seg_{iter}.png'
    path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/SYNTHETIC/{spec}/{split}/SYNTHETIC_{spec}_{split}_{image_id}.png')
    _, axes = plt.subplots(1, len(uris) + 1, figsize=(16, 8))
    img = read_image(path=path_img_orig)
    axes[0].imshow(to_pil_image(img))
    axes[0].axis('off')
    axes[0].set_title('image')
    for i, (method, uri) in enumerate(uris.items()):
        path_img = os.path.join(uri, 'xai', split, file_ann)
        img = read_image(path=path_img)
        axes[i+1].imshow(to_pil_image(img))
        axes[i+1].axis('off')
        axes[i+1].set_title(labels_methods[method])
    plt.show()

### Localization performance at varying CAM thresholds

In [ ]:
experiment_name = 'mwsol3_vgg16_synthetic'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'vgg16',
    'tags.architecture_type': 'cam',
    'tags.dataset_spec': 'd1b',
    'tags.label': 'gap',
    'status': 'FINISHED'
}
boxacc, prcurve = get_mlflow_boxacc_prcurve(
    experiment_name=experiment_name, conditions=conditions, split='val', box_metric='MaxBoxAccV3', mask_metric=True)
# BoxAcc
title = f"VGG16-GAP synthetic BoxAcc"
plot_boxacc(boxacc, title=title, iou_thresholds=[50])
if prcurve is not None:
    # PR Curve
    title = f"VGG16-GAP synthetic PR Curve"
    plot_prcurve(prcurve, title)

## VGG16 vanilla

### Classification versus localization

In [ ]:
metric = 'MaxBoxAccV3'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'gradcam',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, col_wrap=4, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_vgg16_base_cam_synthetic.png')

### MaxBoxAccV3 recall

In [ ]:
metric = 'MaxBoxAccV3_recall'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'base',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

In [ ]:
df = mlflow_get_runs(experiment_name, conditions)
df = df.loc[:, ['tags.method','tags.dataset_spec', 'metrics.test_MaxBoxAccV3_precision', 'metrics.test_MaxBoxAccV3_recall', 'metrics.test_PxAP']]
df.columns = ['method', 'dataset', 'precision', 'recall', 'pxap']
df = df.replace({'method': labels_methods})
df = df.sort_values(['dataset','method'])
df = df.set_index(['method', 'dataset']).unstack(1)
for metric in ['precision', 'recall', 'pxap']:
    print(df.loc[:, [metric]].to_string())

In [ ]:
df = mlflow_get_runs(f'mwsol4_{arch}_{dataset_name.lower()}', conditions)
df = df.loc[:, ['tags.method','tags.dataset_spec', 'metrics.test_MaxBoxAccV3_recall', 'metrics.test_cam_energy_mean', 'metrics.test_cam_energy_std', 'metrics.test_cam_pixels_mean', 'metrics.test_cam_pixels_std']]
df.columns = ['method', 'dataset', 'recall', 'energy_mean', 'energy_std', 'pixels_mean', 'pixels_std']
df = df.replace({'method': labels_methods})
df = df.sort_values(['dataset','method'])
for dataset in sorted(df.loc[:, 'dataset'].unique()):
    print(df.loc[df['dataset'] == dataset].set_index(['method', 'dataset']).unstack(1).to_string())

In [ ]:
df2 = df.loc[:, ['dataset', 'method', 'energy_mean', 'pixels_mean']]
df2.columns = ['dataset','method','energy','pixels']
df2 = df2.set_index(['dataset','method']).stack().reset_index()
df2.columns = ['dataset','method','data','value']
sns.set_style("darkgrid")
hue_order = ['gradcam','gradcam++','scorecam']
# g = sns.FacetGrid(df2, col="data", hue="method", hue_order=hue_order, sharex=True, sharey=False)
# g.map_dataframe(sns.barplot, x="dataset", y="value")
# g.add_legend()
sns_fig = sns.catplot(data=df2, x="dataset", y="value", hue="method", col="data", kind="bar", sharey=False)
sns_fig.savefig(f'fig_vgg16_base_pixel_energy.png')

### MaxBoxAccV3 precision

In [ ]:
metric = 'MaxBoxAccV3_precision'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### PxAP

In [ ]:
metric = 'PxAP'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

In [ ]:
metric = 'MaxBoxAccV3'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'gradcam',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec']
cols_renamed = ['run_id', 'method', 'dataset']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
# sns.set_context("paper")
hue_order = ['accuracy','MaxBoxAccV3']
# sns_plot = sns.relplot(data, x="epochs", y="values", hue="metric", col="dataset", col_wrap=4, kind="line")
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, col_wrap=4, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_vgg16_base_cam_synthetic.png')

### Visual explanations

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image

split = 'test'
image_id = 4
iter = 0
metric = 'MaxBoxAccV3'
metric_name = metric if metric is not None else 'Metrics'
arch = 'vgg16'
arch_type = 'vanilla'
network = 'VGG16'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')

for spec in sorted(df['tags.dataset_spec'].unique()):
    df_spec = df.loc[df['tags.dataset_spec'] == spec]
    df_spec = df_spec.loc[:, ['tags.method', 'tags.dataset_spec', 'artifact_uri']]
    spec_dict = df_spec.set_index('tags.method').sort_index().to_dict()
    uris = spec_dict['artifact_uri']
    spec = '_'.join(list(spec))
    filebase = f'SYNTHETIC_{spec}_test_{image_id}'
    file_img = filebase + f'_img_{iter}.png'
    file_ann = filebase + f'_ann_{iter}.png'
    path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/SYNTHETIC/{spec}/{split}/SYNTHETIC_{spec}_{split}_{image_id}.png')
    _, axes = plt.subplots(1, 4, figsize=(16, 8))
    img = read_image(path=path_img_orig)
    axes[0].imshow(to_pil_image(img))
    axes[0].axis('off')
    axes[0].set_title('image')
    for i, (method, uri) in enumerate(uris.items()):
        path_img = os.path.join(uri, 'xai', split, file_ann)
        img = read_image(path=path_img)
        axes[i+1].imshow(to_pil_image(img))
        axes[i+1].axis('off')
        axes[i+1].set_title(labels_methods[method])
    plt.show()

## ResNet-50

### Classificaton versus Localization

In [ ]:
dataset = 'SYNTHETIC'
spec = 'd1b'
archtype = 'cam'
experiment_name = 'wsol_resnet50_synthetic'
run_name = 'resnet50_minmaxcam_synthetic_d1b'
# run_id = '63fa71e18f4849a19f49b2923adf57a3'
# run_id = 'e6f2d6ba4ca24e31ba7b57c2e2d0266b'
conditions = {'tags.mlflow.runName': run_name}
runs = mlflow_get_runs(experiment_name, conditions=conditions)

In [ ]:
runs

In [ ]:
from mlflow import MlflowClient

run_id = runs['run_id'][0]
client = MlflowClient()
val_accuracy = [metric.value for metric in client.get_metric_history(run_id, 'val_accuracy')]
steps = list(range(len(val_accuracy)))
val_maxboxaccv3 = [metric.value for metric in client.get_metric_history(run_id, 'val_MaxBoxAccV3')]
val_pxap = [metric.value for metric in client.get_metric_history(run_id, 'val_PxAP')]

In [ ]:
plt.plot(steps, val_maxboxaccv3, label = 'MaxBoxAccV3')
plt.plot(steps, val_accuracy, label='Classification accuracy')
plt.xlabel("Training steps")
plt.ylabel("Metric")
plt.title('Classification versus localization performance')
plt.legend()
plt.show()

In [ ]:
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'tags.label': 'gap',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, col_wrap=4, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_resnet50_cam_synthetic.png')

In [ ]:
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.dataset_spec': 'd1b',
    'tags.method': 'cam',
    'tags.label': 'gap',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("whitegrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, hue="metric", hue_order=hue_order, height=4)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
# sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.4, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_resnet50_cam_d1b.png')

In [ ]:
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'wsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'minmaxcam',
    'tags.dataset_spec': 'd1b',
    'tags.train': 'True',
    'run_id': 'df000b8b56444239967892aecd5cc432',
    'status': 'FINISHED'
}

runs = mlflow_get_runs(experiment_name, conditions)
df = runs
cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.4, -0.1), ncol=2, labelspacing=1.0, title=None)

### MaxBoxAccV3 recall

#### non-regularized

In [ ]:
metric = 'MaxBoxAccV3_recall'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'MaxBoxAccV3_recall'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### MaxBoxAccV3 precision

#### non-regularized

In [ ]:
metric = 'MaxBoxAccV3_precision'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'MaxBoxAccV3_precision'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### PxAP

#### non-regularized

In [ ]:
metric = 'PxAP'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'gap',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

#### MinMaxCAM-regularized

In [ ]:
metric = 'PxAP'
metric_name = metric if metric is not None else 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=[metric], caption=caption, label=label)

### Visual explanation examples

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image

image_id = 10
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'SYNTHETIC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol2_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')

speclist = sorted(df['tags.dataset_spec'].unique())
for spec in speclist:
    df_spec = df.loc[df['tags.dataset_spec'] == spec]
    df_spec = df_spec.loc[:, ['tags.method', 'tags.dataset_spec', 'artifact_uri']]
    df_spec = df_spec.sort_values('tags.method')
    spec_dict = df_spec.set_index('tags.method').to_dict()
    uris = spec_dict['artifact_uri']
    spec = '_'.join(list(spec))
    filebase = f'SYNTHETIC_{spec}_test_{image_id}'
    file_img = filebase + '_img_0.png'
    file_ann = filebase + '_ann_0.png'
    file_cam = filebase + '_cam_0.png'
    file_seg = filebase + '_seg_0.png'
    path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/SYNTHETIC/{spec}/{split}/SYNTHETIC_{spec}_{split}_{image_id}.png')
    _, axes = plt.subplots(1, len(uris) + 1, figsize=(16, 8))
    img = read_image(path=path_img_orig)
    axes[0].imshow(to_pil_image(img))
    axes[0].axis('off')
    axes[0].set_title('image')
    for i, (method, uri) in enumerate(uris.items()):
        path_img = os.path.join(uri, 'xai', split, file_ann)
        img = read_image(path=path_img)
        axes[i+1].imshow(to_pil_image(img))
        axes[i+1].axis('off')
        axes[i+1].set_title(labels_methods[method])
    plt.show()

# Localization ImageNet dataset

## VGG16-GAP

### Localization metrics

In [ ]:
metrics = None
metric_name = 'Metrics'
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=metrics, caption=caption, label=label)

### Visual explanations

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image

# image_no_list = list(range(1,21))
image_no_list = [1, 15]
# image_no_list = [1,2,3,4,8]
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
iter = 0
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
df = df.loc[df['tags.method'] != 'minmaxcam'] # filter out minmaxcam network was not trained sufficiently
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')

speclist = sorted(df['tags.dataset_spec'].unique())
for spec in speclist:
    df_spec = df.loc[df['tags.dataset_spec'] == spec]
    df_spec = df_spec.loc[:, ['tags.method', 'tags.dataset_spec', 'artifact_uri',
                              'metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50']]
    df_spec = df_spec.sort_values('tags.method')
    spec_dict = df_spec.set_index('tags.method').to_dict()
    uris = spec_dict['artifact_uri']
    opt_thresholds = list(spec_dict['metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50'].values())
    spec = '_'.join(list(spec))
    fig, axes = plt.subplots(len(image_no_list), len(uris) + 1, figsize=(16, 8))
    for row, image_no in enumerate(image_no_list):
        filebase = f'ILSVRC2012_{split}_{image_no:0>8}'
        file_img = filebase + f'_img_{iter}.png'
        file_ann = filebase + f'_ann_{iter}.png'
        file_cam = filebase + f'_cam_{iter}.png'
        file_seg = filebase + f'_seg_{iter}.png'
        image_id = f'{split}/ILSVRC2012_{split}_{image_no:0>8}.JPEG'
        path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/ILSVRC/{image_id}')
        img = read_image(path=path_img_orig)
        axes[row, 0].imshow(to_pil_image(img))
        axes[row, 0].axis('off')
        class_id = class_labels[image_id]
        label_name = label_names[str(class_id)]
        axes[row, 0].set_title(f'image ({label_name})')
        for col, (method, uri) in enumerate(uris.items()):
            path_img = os.path.join(uri, 'xai', split, file_ann)
            img = read_image(path=path_img)
            axes[row, col+1].imshow(to_pil_image(img))
            axes[row, col+1].axis('off')
            title = f"{labels_methods[method]} (" + r'$\tau^*' + f' = {opt_thresholds[i]}$)'
            axes[row, col+1].set_title(title)
    plt.tight_layout()
    plt.savefig(f'fig_explain_vgg16_gap_imagenet.png', dpi=fig.dpi)
    plt.show()

### Classification versus Localization

In [ ]:
arch = 'vgg16'
arch_type = 'cam'
network = 'VGG16-GAP'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'wsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset_spec', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
# sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_vgg16_gap_cam_imagenet.png')

## ResNet-50

### Classification versus localization accuracy

In [ ]:
df

In [ ]:
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'minmaxcam',
    'tags.train': 'True',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)

cols_mlflow = ['run_id', 'tags.method', 'tags.dataset', 'metrics.val_accuracy', 'metrics.val_MaxBoxAccV3', 'start_time']
cols_renamed = ['run_id', 'method', 'dataset', 'accuracy', 'MaxBoxAccV3', 'start_time']
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
df = df.drop(columns='start_time')
df = df.assign(epochs=epoch_steps, accuracy=epoch_val_accuracy, MaxBoxAccV3=epoch_maxboxaccv3)
df = df.explode(column=['epochs', 'accuracy', 'MaxBoxAccV3'])
df = df.drop(columns='run_id').set_index(['method','dataset','epochs']).stack().reset_index()
df.columns = ['method','dataset','epochs','metric','values']

data = df.sort_values('dataset')

sns.set_style("darkgrid")
hue_order = ['accuracy','MaxBoxAccV3']
g = sns.FacetGrid(data, col="dataset", hue="metric", hue_order=hue_order, sharex=True, sharey=True)
g.map_dataframe(sns.lineplot, x="epochs", y="values")
g.add_legend()
sns.move_legend(g, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=2, labelspacing=1.0)
g.savefig(f'fig_loc_vs_acc_resnet50_cam_synthetic.png')

### Localization metrics

#### Non-regularized

In [ ]:
metrics = ['MaxBoxAcc', 'MaxBoxAccV2', 'MaxBoxAccV3_precision', 'MaxBoxAccV3_recall']
metric_name = 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
runs = mlflow_get_runs(experiment_name, conditions)
runs = runs.loc[runs.start_time > '2023-05-01']
runs = runs.loc[runs['tags.method']!='minmaxcam']
dataframe_to_latex(runs, split=split, metrics=metrics, label=label, caption=caption)

#### MinMaxCAM-regularized

In [ ]:
metrics = None
metric_name = 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
caption=f'{metric_name} for {network} on {dataset_name}'
label=f'tb:{metric_name}_{arch}_{dataset_name}'.lower()
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
mlflow_runs_to_latex(experiment_name, conditions=conditions, split=split, metrics=metrics, caption=caption, label=label)

### Visual explanations non-regularized

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image

image_no_list = list(range(1,21))
# image_no_list = [1, 15]
# image_no_list = [2,3,4,8]
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
iter = 0
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
dfv = mlflow_get_runs(experiment_name, conditions)
dfv = dfv.loc[dfv.start_time > '2023-05-01'] # use newest experiment runs
dfv = dfv.loc[dfv['tags.method'] != 'minmaxcam'] # non-regularized
dfv.loc[:, 'artifact_uri'] = dfv['artifact_uri'].str.removeprefix('file://')

dfv = dfv.loc[:, ['tags.method', 'artifact_uri', 'metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50']]
dfv = dfv.sort_values('tags.method')
dfv_dict = dfv.set_index('tags.method').to_dict()
uris_vanilla = dfv_dict['artifact_uri']
opt_thresholds_vanilla = list(dfv_dict['metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50'].values())

experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'status': 'FINISHED'
}
dfm = mlflow_get_runs(experiment_name, conditions)
dfm = dfm.loc[dfm.start_time > '2023-05-01'] # use newest experiment runs
dfm = dfm.loc[dfm['tags.method'] != 'minmaxcam'] # non-regularized
dfm.loc[:, 'artifact_uri'] = dfm['artifact_uri'].str.removeprefix('file://')

dfm = dfm.loc[:, ['tags.method', 'artifact_uri', 'metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50']]
dfm = dfm.sort_values('tags.method')
dfm_dict = dfm.set_index('tags.method').to_dict()
uris_minmaxcam = dfm_dict['artifact_uri']
opt_thresholds_minmaxcam = list(dfm_dict['metrics.val_MaxBoxAccV3_optimal_threshold_IOU_50'].values())

uris_list = [uris_vanilla, uris_minmaxcam]
opt_thresholds_list = [opt_thresholds_vanilla, opt_thresholds_minmaxcam]
regularization_list = ['Vanilla', 'MinMaxCAM']
for image_no in image_no_list:
    fig, axes = plt.subplots(2, len(uris_vanilla) + 1, figsize=(16, 8))
    for row, (uris, opt_thresholds, regularization) in enumerate(zip(uris_list, opt_thresholds_list, regularization_list)):
        filebase = f'ILSVRC2012_{split}_{image_no:0>8}'
        file_img = filebase + f'_img_{iter}.png'
        file_ann = filebase + f'_ann_{iter}.png'
        file_cam = filebase + f'_cam_{iter}.png'
        file_seg = filebase + f'_seg_{iter}.png'
        image_id = f'{split}/ILSVRC2012_{split}_{image_no:0>8}.JPEG'
        path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/ILSVRC/{image_id}')
        img = read_image(path=path_img_orig)
        axes[row, 0].imshow(to_pil_image(img))
        axes[row, 0].axis('off')
        class_id = class_labels[image_id]
        label_name = label_names[str(class_id)]
        axes[row, 0].set_title(f'image ({label_name})')
        for col, (method, uri) in enumerate(uris.items()):
            path_img = os.path.join(uri, 'xai', split, file_ann)
            img = read_image(path=path_img)
            axes[row, col+1].imshow(to_pil_image(img))
            axes[row, col+1].axis('off')
            title = f"{labels_methods[method]} (" + r'$\tau^*' + ' = ' + f'{opt_thresholds[col]:.2f}$) | {regularization}'
            axes[row, col+1].set_title(title)
    plt.tight_layout()
    # plt.savefig(f'fig_explain_resnet50_imagenet.png', dpi=fig.dpi)
    plt.show()

### Visual explanation non-regularized vs MinMaxCam-regularized

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
import pandas as pd

image_no_list = list(range(1,21))
# image_no_list = [1,2,3,4,8]
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol_{arch}_{dataset_name.lower()}'
split = 'val' if dataset == 'ILSVRC' else 'test'
iter = 0
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
df = df.loc[df.start_time > '2023-05-01']
df = df.loc[df['tags.method'] != 'minmaxcam']

experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'tags.label': 'minmaxcam',
    'status': 'FINISHED'
}
df2 = mlflow_get_runs(experiment_name, conditions)
df2 = df2.replace({'tags.method': {'cam':'minmaxcam'}})
df = pd.concat([df, df2], axis=0, ignore_index=True)

df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')
df = df.loc[:, ['tags.method', 'artifact_uri']]
df = df.sort_values('tags.method')
df_dict = df.set_index('tags.method').to_dict()
uris = df_dict['artifact_uri']
for image_no in image_no_list:
    filebase = f'ILSVRC2012_{split}_{image_no:0>8}'
    file_img = filebase + f'_img_{iter}.png'
    file_ann = filebase + f'_ann_{iter}.png'
    file_cam = filebase + f'_cam_{iter}.png'
    file_seg = filebase + f'_seg_{iter}.png'
    image_id = f'{split}/ILSVRC2012_{split}_{image_no:0>8}.JPEG'
    path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/ILSVRC/{image_id}')
    _, axes = plt.subplots(1, len(uris) + 1, figsize=(16, 8))
    img = read_image(path=path_img_orig)
    axes[0].imshow(to_pil_image(img))
    axes[0].axis('off')
    class_id = class_labels[image_id]
    label_name = label_names[str(class_id)]
    axes[0].set_title(f'image ({label_name})')
    for i, (method, uri) in enumerate(uris.items()):
        path_img = os.path.join(uri, 'xai', split, file_ann)
        img = read_image(path=path_img)
        axes[i+1].imshow(to_pil_image(img))
        axes[i+1].axis('off')
        axes[i+1].set_title(labels_methods[method])
    plt.show()

### Localization in function of object instances

In [ ]:
import munch
import pandas as pd
from src.data_loaders import get_bounding_boxes

def mch(**kwargs):
    return munch.Munch(dict(**kwargs))

def configure_metadata(metadata_root):
    metadata = mch()
    metadata.image_ids = os.path.join(metadata_root, 'image_ids.txt')
    metadata.image_ids_proxy = os.path.join(metadata_root,
                                            'image_ids_proxy.txt')
    metadata.class_labels = os.path.join(metadata_root, 'class_labels.txt')
    metadata.image_sizes = os.path.join(metadata_root, 'image_sizes.txt')
    metadata.localization = os.path.join(metadata_root, 'localization.txt')
    return metadata

In [ ]:
data_root = '../data/dataset'
metadata_root = '../data/metadata'
dataset_list = ['CUB', 'ILSVRC', 'SYNTHETIC']
_SPLITS = ('train', 'val', 'test')
metadata_path = os.path.join(metadata_root, 'ILSVRC')
metadata_splits = {split: configure_metadata(os.path.join(metadata_path, split)) for split in _SPLITS}

bboxes_dict = get_bounding_boxes(metadata_splits['val'])
bb_val = {}
num_to_imgids = {}
for image_id, bboxes in bboxes_dict.items():
    instances = len(bboxes)
    if instances in bb_val:
        bb_val[instances] += 1
        num_to_imgids[instances].append(image_id)
    else:
        bb_val[instances] = 1
        num_to_imgids[instances] = [image_id]
num_to_imgids = dict(sorted(num_to_imgids.items()))
bb_val = dict(sorted(bb_val.items()))
bb_dict = {'instances': bb_val.keys(), 'images': bb_val.values()}
df_bb = pd.DataFrame(bb_dict)
total_images = np.sum(df_bb.loc[:, 'images'])
df_bb.loc[:, 'fraction'] = df_bb.loc[:, 'images'].div(total_images)
df_bb

In [ ]:
metrics = None
metric_name = 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}_inst'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions=conditions)
cols = ['tags.method', 'tags.filter_instances', f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'instances', 'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df.loc[:, 'instances'] = df.instances.astype(int)
df = df.merge(df_bb, on='instances', how='inner')
df = df.sort_values(['instances'], ascending=[False])
sns.set_style("darkgrid")
sns.set_context("paper")
ax = sns.barplot(data=df, y="instances", x="recall", color='blue', orient='h', order=df['instances'])
ax.bar_label(ax.containers[0], fmt=' %.2f')
ax.set_xlim(0, 100)
plt.savefig(f'fig_resnet50_imagenet_inst_recall.png')

In [ ]:
df.sort_values('instances', ascending=True)

### Localization performance at varying CAM thresholds

In [ ]:
metrics = None
metric_name = 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}_inst'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions=conditions)
cols = ['tags.method', 'tags.filter_instances', f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'instances', 'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df.loc[:, 'instances'] = df.instances.astype(int)
df = df.merge(df_bb, on='instances', how='inner')
df = df.sort_values(['instances'], ascending=[False])
sns.set_style("darkgrid")
sns.set_context("paper")
ax = sns.barplot(data=df, y="instances", x="precision", color='blue', orient='h', order=df['instances'])
ax.bar_label(ax.containers[0], fmt=' %.2f')
ax.set_xlim(0, 100)
plt.savefig(f'fig_resnet50_imagenet_inst_precision.png')

In [ ]:
metrics = None
metric_name = 'Metrics'
arch = 'resnet50'
arch_type = 'cam'
network = 'ResNet-50'
dataset = 'ILSVRC'
dataset_name = 'ImageNet' if dataset == 'ILSVRC' else 'synthetic'
experiment_name = f'mwsol3_{arch}_{dataset_name.lower()}_inst'
split = 'val' if dataset == 'ILSVRC' else 'test'
conditions = {
    'tags.dataset': dataset,
    'tags.architecture': arch,
    'tags.architecture_type': arch_type,
    'tags.method': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions=conditions)
cols = ['tags.method', 'tags.filter_instances', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'instances', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall']:
    df[metric] *= 100 # to percentage
df.loc[:, 'instances'] = df.instances.astype(int)
df = df.merge(df_bb, on='instances', how='inner')
df = df.loc[:, ['instances', 'precision', 'recall']]
df = df.sort_values(['instances'])
df = df.set_index('instances').stack().reset_index()
df.columns = ['instances', 'metric', 'values']
sns.set_style("darkgrid")
sns.set_context("paper")
ax = sns.barplot(data=df, y="instances", x="values", hue='metric', hue_order=['precision','recall'], orient='h')
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f')
ax.set_xlim(0, 100)

In [ ]:
experiment_name = 'mwsol_resnet50_imagenet'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
boxacc, prcurve = get_mlflow_boxacc_prcurve(experiment_name=experiment_name, conditions=conditions, split='val', box_metric='MaxBoxAccV3', mask_metric=None)
# BoxAcc
title = f"ResNet-50 ImageNet BoxAcc"
plot_boxacc(boxacc, title=title, iou_thresholds=[50])
if prcurve is not None:
    # PR Curve
    title = f"ResNet-50 ImageNet PR Curve"
    plot_prcurve(prcurve, title)

# Iterative localization

## VGG16-GAP synthetic

In [ ]:
experiment_name = 'mwsol_vgg16_synthetic_iter'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.dataset_spec': 'd4b',
    'tags.architecture': 'vgg16',
    'tags.architecture_type': 'cam',
    'tags.method': 'cam',
    'status': 'FINISHED'
}
# conditions |= {
#     'params.bbox_mask_strategy': 'random',
#     'params.bbox_merge_strategy': 'add',
#     'params.iter_stop_prob_delta': '0.25',
# }
df_runs = mlflow_get_runs(experiment_name, conditions)

In [ ]:
import itertools
import numpy as np

label = 'tab:iter_metrics_vgg16_cam_synthetic'
caption = 'Iterative CAM localization metrics for VGG16-GAP on synthetic dataset'
metrics = ['MaxBoxAccV3_recall', 'MaxBoxAccV3_precision', 'MaxBoxAccV3_f1']
metrics_short = ['recall','precision','f1']
split = 'test'

df = df_runs
cols_metrics_mlflow = [f'metrics.{split}_{metric}' for metric in metrics]
cols_params = ['bbox_mask_strategy', 'bbox_merge_strategy', 'iter_stop_prob_delta']
cols_params_mlflow = [f'params.{p}' for p in cols_params]
cols_mlflow = ['tags.method', 'tags.architecture', 'tags.architecture_type', 'tags.dataset', 'tags.dataset_spec']
cols_mlflow += cols_metrics_mlflow + cols_params_mlflow
cols_renamed = [c.split('.')[-1] for c in cols_mlflow]
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
mapper = dict([(f'{split}_{metric}', metric) for metric in metrics])
df = df.rename(columns=mapper)
mapper = dict(zip(metrics, metrics_short))
df = df.rename(columns=mapper)
cols_params_short = ['mask', 'merge', 'stop']
mapper = dict(zip(cols_params, cols_params_short))
df = df.rename(columns=mapper)
# df.loc[:, 'dataset'] = df['dataset'].str.cat(df['dataset_spec'], sep='\_').str.lower()
# pd.set_option("display.precision", 2)
metrics = metrics_short
for metric in metrics:
    df[metric] *= 100 # to percentage
cols_filter = cols_params_short + metrics
df = df.loc[:, cols_filter]
cols_pivot = ['merge']
cols_index = ['stop', 'mask']
pivot_values = [np.sort(df.loc[:, c].unique()).tolist() for c in cols_pivot]
df = df.pivot(index=cols_index, columns=cols_pivot, values=metrics)
iterables = [metrics]
cols_format = dict([(col_index, '{:.2f}') for col_index in itertools.product(metrics, *pivot_values)])
s = df.style
s = s.format(cols_format)
s = s.highlight_max(subset=metrics, axis=0, props='color:{teal}; bfseries: ;')
s = s.highlight_min(subset=metrics, axis=0, props='color:{purple}; bfseries: ;')
latex_table = s.to_latex(
        # column_format="llll" + "r"*len(metrics),
        position="ht", position_float="centering",
        hrules=False, label=label, caption=caption,
        multirow_align="t", multicol_align="c")
print(latex_table)

In [ ]:
import itertools
import numpy as np

label = 'tab:iter_metrics_vgg16_cam_synthetic'
caption = 'Iterative CAM localization metrics for VGG16-GAP on synthetic dataset'
metrics = ['MaxBoxAccV3_recall', 'MaxBoxAccV3_precision', 'MaxBoxAccV3_f1']
metrics_short = ['recall','precision','f1']
split = 'test'

df = df_runs
cols_metrics_mlflow = [f'metrics.{split}_{metric}' for metric in metrics]
cols_params = ['bbox_mask_strategy', 'bbox_merge_strategy', 'iter_stop_prob_delta']
cols_params_mlflow = [f'params.{p}' for p in cols_params]
cols_mlflow = ['tags.method', 'tags.architecture', 'tags.architecture_type', 'tags.dataset', 'tags.dataset_spec']
cols_mlflow += cols_metrics_mlflow + cols_params_mlflow
cols_renamed = [c.split('.')[-1] for c in cols_mlflow]
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)
mapper = dict([(f'{split}_{metric}', metric) for metric in metrics])
df = df.rename(columns=mapper)
mapper = dict(zip(metrics, metrics_short))
df = df.rename(columns=mapper)
cols_params_short = ['mask', 'merge', 'stop']
mapper = dict(zip(cols_params, cols_params_short))
df = df.rename(columns=mapper)
# df.loc[:, 'dataset'] = df['dataset'].str.cat(df['dataset_spec'], sep='\_').str.lower()
# pd.set_option("display.precision", 2)
metrics = metrics_short
for metric in metrics:
    df[metric] *= 100 # to percentage
cols_filter = cols_params_short + metrics
df = df.loc[:, cols_filter]
cols_pivot = ['merge']
cols_index = ['stop', 'mask']
pivot_values = [np.sort(df.loc[:, c].unique()).tolist() for c in cols_pivot]
df = df.pivot(index=cols_index, columns=cols_pivot, values=metrics)
cols_format = dict([(col_index, '{:.2f}') for col_index in itertools.product(metrics, *pivot_values)])
s = df.style
s = s.format(cols_format)
s = s.highlight_max(subset=metrics, axis=0, props='color:{teal}; bfseries: ;')
s = s.highlight_min(subset=metrics, axis=0, props='color:{purple}; bfseries: ;')
latex_table = s.to_latex(
        # column_format="llll" + "r"*len(metrics),
        position="ht", position_float="centering",
        hrules=False, label=label, caption=caption,
        multirow_align="t", multicol_align="c")
print(latex_table)

In [ ]:
experiment_name = 'mwsol_vgg16_synthetic_iter'
split = 'test'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'vgg16',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
# conditions |= {'tags.dataset_spec': 'd4b'}
# conditions |= {
#     'params.bbox_mask_strategy': 'mean',
#     'params.bbox_merge_strategy': 'unify',
#     'params.iter_stop_prob_delta': '0.5',
# }

df = mlflow_get_runs(experiment_name, conditions)

label = 'tab:iter_metrics_vgg16_cam_synthetic'
caption = 'Iterative localization metrics for VGG16-GAP on synthetic dataset'
cols_metrics = ['MaxBoxAccV3_recall', 'MaxBoxAccV3_precision', 'MaxBoxAccV3_f1']
cols_metrics_mlflow = [f'metrics.{split}_{metric}' for metric in cols_metrics]
cols_metrics_short = ['recall','precision','f1']
cols_params = ['bbox_mask_strategy', 'bbox_merge_strategy', 'iter_stop_prob_delta']
cols_params_mlflow = [f'params.{p}' for p in cols_params]
cols_params_short = ['mask', 'merge', 'stop']
cols_tags_mlflow = ['tags.method', 'tags.dataset_spec']
cols_tags_short = ['method', 'dataset']

cols_mlflow = cols_tags_mlflow + cols_params_mlflow + cols_metrics_mlflow
cols_renamed = cols_tags_short + cols_params_short + cols_metrics_short
mapper = dict(zip(cols_mlflow, cols_renamed))
df = df.loc[:, cols_mlflow].rename(columns=mapper)

metrics = cols_metrics_short
for metric in metrics:
    df[metric] *= 100 # to percentage

In [ ]:
# baseline without iteration
experiment_name = 'mwsol3_vgg16_synthetic'#'mwsol_vgg16_synthetic'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'vgg16',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'test'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[((df_base['tags.method'] != "minmaxcam") & (df_base['tags.label'] == "gap")) | (df_base['tags.method'] == "minmaxcam")]
cols = ['tags.method', 'tags.dataset_spec',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
prec_base = df_base.set_index(['spec', 'method']).to_dict()

# iterative experiments
experiment_name = 'mwsol_vgg16_synthetic_iter'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'vgg16',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
# conditions |= {
#     'params.bbox_mask_strategy': 'random',
#     'params.bbox_merge_strategy': 'add',
#     'params.iter_stop_prob_delta': '0.25',
# }
df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method', 'tags.dataset_spec',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['method', 'spec'])
for spec in [f'd{i}b' for i in range(1,5)]:
    df2 = df.loc[(df['spec'] == spec)]
    sns.set_style("darkgrid")
    sns.set_context("paper")
    markers = {'drop': 'v', 'add': '^', 'unify':'o'}
    sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
    size_order = ['0.25', '0.5', '1.0']
    hue_order = ['mean','random','zero']
    sns_fig = sns.relplot(data=df2, x="recall", y="precision", hue="mask", style="merge",
                          size="stop", size_order=size_order, col='method', col_wrap=2,
                          markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
    sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)

    # flatten axes into a 1-d array
    axes = sns_fig.axes.flatten()
    # iterate through the axes
    methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
    for i, ax in enumerate(axes):
        method = methods[i]
        precision = prec_base['precision'][(spec, method)] * 100
        recall = prec_base['recall'][(spec, method)] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if i == 1:
            lines = [line_prec, line_rec]
            labels = ['precision', 'recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
    sns_fig.savefig(f'fig_iter_vgg16_gap_syn_{spec}.png')

## ResNet-50 synthetic

### Oracle-based iterative localization

In [ ]:
# baseline without iteration
experiment_name = 'mwsol3_resnet50_synthetic'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'test'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[
    ((df_base['tags.method'] != "minmaxcam") & (df_base['tags.label'] == "gap")) | (df_base['tags.method'] == "minmaxcam")]
cols = ['tags.method', 'tags.dataset_spec',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
# replace gradcam method by cam: localization results are identical for resnet50 but in the iter dataset, cam is used.
# df_base = df_base.replace({'method': {'gradcam':'cam'}})
prec_base = df_base.set_index(['spec', 'method']).to_dict()

# iterative experiments
experiment_name = 'mwsol3_resnet50_synthetic_iter'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
# conditions |= {
#     'params.bbox_mask_strategy': 'random',
#     'params.bbox_merge_strategy': 'add',
#     'params.iter_stop_prob_delta': '0.25',
# }
df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method', 'tags.dataset_spec',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['method', 'spec'])
for spec in [f'd{i}b' for i in range(1,5)]:
    df2 = df.loc[(df['spec'] == spec)]
    sns.set_style("darkgrid")
    sns.set_context("paper")
    markers = {'drop': 'v', 'add': '^', 'unify':'o'}
    sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
    size_order = ['0.25', '0.5', '1.0']
    hue_order = ['mean','random','zero']
    sns_fig = sns.relplot(data=df2, x="recall", y="precision", hue="mask", style="merge",
                          size="stop", size_order=size_order, col='method', col_wrap=2,
                          markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
    sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)

    # flatten axes into a 1-d array
    axes = sns_fig.axes.flatten()
    # iterate through the axes
    methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
    for i, ax in enumerate(axes):
        method = methods[i]
        precision = prec_base['precision'][(spec, method)] * 100
        recall = prec_base['recall'][(spec, method)] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if i == 1:
            lines = [line_prec, line_rec]
            labels = ['precision', 'recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
    sns_fig.savefig(f'fig_iter_resnet50_syn_{spec}.png')

### Score map based iterative localization

In [ ]:
# baseline without iteration
experiment_name = 'mwsol3_resnet50_synthetic'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'test'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[
    ((df_base['tags.method'] != "minmaxcam") & (df_base['tags.label'] == "gap")) | (df_base['tags.method'] == "minmaxcam")]
cols = ['tags.method', 'tags.dataset_spec',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
# replace gradcam method by cam: localization results are identical for resnet50 but in the iter dataset, cam is used.
# df_base = df_base.replace({'method': {'gradcam':'cam'}})
prec_base = df_base.set_index(['spec', 'method']).to_dict()

# iterative experiments
experiment_name = 'mwsol4_resnet50_synthetic_iter_mask_cam'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method', 'tags.dataset_spec',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['method', 'spec'])
for spec in [f'd{i}b' for i in range(1,5)]:
    df2 = df.loc[(df['spec'] == spec)]
    sns.set_style("darkgrid")
    sns.set_context("paper")
    markers = {'drop': 'v', 'add': '^', 'unify':'o'}
    sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
    size_order = ['0.25', '0.5', '1.0']
    hue_order = ['mean','random','zero']
    sns_fig = sns.relplot(data=df2, x="recall", y="precision", hue="mask", style="merge",
                          size="stop", size_order=size_order, col='method', col_wrap=2,
                          markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
    sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)

    # flatten axes into a 1-d array
    axes = sns_fig.axes.flatten()
    # iterate through the axes
    methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
    for i, ax in enumerate(axes):
        method = methods[i]
        precision = prec_base['precision'][(spec, method)] * 100
        recall = prec_base['recall'][(spec, method)] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if i == 1:
            lines = [line_prec, line_rec]
            labels = ['precision', 'recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
    sns_fig.savefig(f'fig_iter_mask_cam_resnet50_syn_{spec}.png')

### bbox_merge_iou_threshold evaluation

In [ ]:
df['stop'].unique()

In [ ]:

df.loc[(df['merge']=='add')&(df['mask']=='random')&(df['stop']=='1.0')&(df['spec']=='d1b')]

In [ ]:
# baseline without iteration
experiment_name = 'mwsol3_resnet50_synthetic'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'test'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[
    ((df_base['tags.method'] != "minmaxcam") & (df_base['tags.label'] == "gap")) | (df_base['tags.method'] == "minmaxcam")]
cols = ['tags.method', 'tags.dataset_spec',
        f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
# replace gradcam method by cam: localization results are identical for resnet50 but in the iter dataset, cam is used.
# df_base = df_base.replace({'method': {'gradcam':'cam'}})
prec_base = df_base.set_index(['spec', 'method']).to_dict()

# iterative experiments
experiment_name = 'mwsol4_resnet50_synthetic_iter_iou'
conditions = {
    'tags.dataset': 'SYNTHETIC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method', 'tags.dataset_spec',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta', 'params.bbox_merge_iou_threshold',
        f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'spec',
                'mask', 'merge', 'stop', 'iou',
                'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['iou', 'spec'])
for spec in [f'd{i}b' for i in range(1,5)]:
    df2 = df.loc[(df['spec'] == spec)]
    sns.set_style("darkgrid")
    sns.set_context("paper")
    markers = {'drop': 'v', 'add': '^', 'unify':'o'}
    sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
    size_order = ['0.25', '0.5', '1.0']
    hue_order = ['mean','random','zero']
    sns_fig = sns.relplot(data=df2, x="recall", y="precision", hue="mask", style="merge",
                          size="stop", size_order=size_order, col='iou', col_wrap=2,
                          markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
    sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)

    # flatten axes into a 1-d array
    axes = sns_fig.axes.flatten()
    # iterate through the axes
    methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
    for i, ax in enumerate(axes):
        method = 'cam'#methods[i]
        precision = prec_base['precision'][(spec, method)] * 100
        recall = prec_base['recall'][(spec, method)] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if i == 1:
            lines = [line_prec, line_rec]
            labels = ['precision', 'recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
    sns_fig.savefig(f'fig_iter_resnet50_syn_iou_{spec}.png')

## ResNet-50 ImageNet

### Oracle-based iterative localization

In [ ]:
# baseline without iteration
experiment_name = 'mwsol_resnet50_imagenet'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'val'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[df_base.start_time > '2023-05-01']
cols = ['tags.method',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
prec_base = df_base.set_index(['method']).to_dict()

# iterative experiments
experiment_name = 'mwsol_resnet50_imagenet_iter'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
# conditions |= {
#     'params.bbox_mask_strategy': 'random',
#     'params.bbox_merge_strategy': 'add',
#     'params.iter_stop_prob_delta': '0.25',
# }
df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['method'])

sns.set_style("darkgrid")
sns.set_context("paper")
markers = {'drop': 'v', 'add': '^', 'unify':'o'}
# markers = {'drop': 'x', 'add': '+', 'unify':'2'}
sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
size_order = ['0.25', '0.5', '1.0']
hue_order = ['mean','random','zero']
sns_fig = sns.relplot(data=df, x="recall", y="precision", hue="mask", style="merge",
                      size="stop", size_order=size_order, col='method', col_wrap=2,
                      markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)
# flatten axes into a 1-d array
axes = sns_fig.axes.flatten()
# iterate through the axes
methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
for i, ax in enumerate(axes):
    method = methods[i]
    precision = prec_base['precision'][method] * 100
    recall = prec_base['recall'][method] * 100
    line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
    line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
    if i == 1:
        lines = [line_prec, line_rec]
        labels = ['precision', 'recall']
        legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
sns_fig.savefig(f'fig_iter_mask_cam_resnet50_imagenet.png')

### Score map based iterative localization

In [ ]:
# baseline without iteration
experiment_name = 'mwsol_resnet50_imagenet'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
split = 'val'
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[df_base.start_time > '2023-05-01']
cols = ['tags.method',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
prec_base = df_base.set_index(['method']).to_dict()

# iterative experiments
experiment_name = 'mwsol4_resnet50_imagenet_iter_mask_cam'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}

df = mlflow_get_runs(experiment_name, conditions)
cols = ['tags.method',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall']
mapper = dict(zip(cols, cols_renamed))
df = df.loc[:, cols].rename(columns=mapper)
df = df.replace({'method': labels_methods})
for metric in ['precision', 'recall', 'f1']:
    df[metric] *= 100 # to percentage
df = df.sort_values(['method'])

sns.set_style("darkgrid")
sns.set_context("paper")
markers = {'drop': 'v', 'add': '^', 'unify':'o'}
# markers = {'drop': 'x', 'add': '+', 'unify':'2'}
sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
size_order = ['0.25', '0.5', '1.0']
hue_order = ['mean','random','zero']
sns_fig = sns.relplot(data=df, x="recall", y="precision", hue="mask", style="merge",
                      size="stop", size_order=size_order, col='method', col_wrap=2,
                      markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
sns.move_legend(sns_fig, loc='center right', bbox_to_anchor=(1.02, 0.5), labelspacing=1.0, frameon=True)
# flatten axes into a 1-d array
axes = sns_fig.axes.flatten()
# iterate through the axes
methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
for i, ax in enumerate(axes):
    method = methods[i]
    precision = prec_base['precision'][method] * 100
    recall = prec_base['recall'][method] * 100
    line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
    line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
    if i == 1:
        lines = [line_prec, line_rec]
        labels = ['precision', 'recall']
        legend = ax.legend(lines, labels, bbox_to_anchor=(1.02, 1.0), loc='upper left', title='baseline', frameon=True)
sns_fig.savefig(f'fig_iter_resnet50_imagenet.png')

### Iterative localization side-by-sice comparision (oracle vs score map)

In [ ]:
# baseline without iteration
experiment_name = 'mwsol_resnet50_imagenet'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[df_base.start_time > '2023-05-01']
cols = ['tags.method',
        'params.bbox_merge_strategy', 'params.bbox_mask_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'merge', 'mask', 'stop', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
prec_base = df_base.set_index(['method']).to_dict()

split = 'val'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
cols = ['tags.method',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall',
        'artifact_uri']
cols_renamed = ['method',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall', 'artifact_uri']
df_list = []
for mask_type, experiment_name in [('oracle based', 'mwsol_resnet50_imagenet_iter'),
                                   ('score map based', 'mwsol4_resnet50_imagenet_iter_mask_cam')]:
    df = mlflow_get_runs(experiment_name, conditions)
    mapper = dict(zip(cols, cols_renamed))
    df = df.loc[:, cols].rename(columns=mapper)
    df = df.replace({'method': labels_methods})
    df = df.loc[df.method != 'Score-CAM'] # not available for score map based i.l.
    for metric in ['precision', 'recall', 'f1']:
        df[metric] *= 100 # to percentage
    df.loc[:, 'type'] = mask_type
    df_list.append(df)

# concatenate dataframes
df = pd.concat(df_list, axis=0, ignore_index=True)
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')
df = df.sort_values(['method', 'mask'])

# plots
sns.set_style("darkgrid")
sns.set_context("paper")
markers = {'drop': 'v', 'add': '^', 'unify':'o'}
sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
size_order = ['0.25', '0.5', '1.0']
hue_order = ['mean','random','zero']
sns_fig = sns.relplot(data=df, x="recall", y="precision", hue="mask", style="merge",
                      size="stop", size_order=size_order, col='method', row='type',
                      markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
sns.move_legend(sns_fig, loc='lower center', bbox_to_anchor=(0.6, 1.0), labelspacing=1.0, frameon=True, ncol=12)
# flatten axes into a 1-d array
# axes = sns_fig.axes.flatten()
# iterate through the axes
row_num, col_num = sns_fig.axes.shape
methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
for r in range(row_num):
    for c in range(col_num):
        ax = sns_fig.axes[r,c]
        method = methods[c]
        precision = prec_base['precision'][method] * 100
        recall = prec_base['recall'][method] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if r == 0 and c == 0:
            lines = [line_prec, line_rec]
            labels = ['baseline precision', 'baseline recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(0.05, 1.075), loc='lower left', title=None, frameon=True, ncols=2)
sns_fig.savefig(f'fig_iter_mask_bbox_vs_cam_resnet50_imagenet.png')

In [ ]:
df.sort_values(['method', 'merge', 'stop','mask', 'type'])

### Iterative localization ImageNet split by instances (oracle vs score map)

In [ ]:
# baseline without iteration
experiment_name = 'mwsol_resnet50_imagenet'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
df_base = mlflow_get_runs(experiment_name, conditions)
df_base = df_base.loc[df_base.start_time > '2023-05-01']
cols = ['tags.method',
        'params.bbox_merge_strategy', 'params.bbox_mask_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall']
cols_renamed = ['method', 'merge', 'mask', 'stop', 'f1', 'precision', 'recall']
df_base = df_base.loc[:, cols]
df_base = df_base.rename(columns=dict(zip(cols, cols_renamed)))
prec_base = df_base.set_index(['method']).to_dict()

split = 'val'
conditions = {
    'tags.dataset': 'ILSVRC',
    'tags.architecture': 'resnet50',
    'tags.architecture_type': 'cam',
    'status': 'FINISHED'
}
cols = ['tags.method',
        'params.bbox_mask_strategy', 'params.bbox_merge_strategy', 'params.iter_stop_prob_delta',
        f'metrics.{split}_MaxBoxAccV3_f1', f'metrics.{split}_MaxBoxAccV3_precision', f'metrics.{split}_MaxBoxAccV3_recall',
        'artifact_uri']
cols_renamed = ['method',
                'mask', 'merge', 'stop',
                'f1', 'precision', 'recall', 'artifact_uri']
df_list = []
for mask_type, experiment_name in [('oracle based', 'mwsol_resnet50_imagenet_iter'),
                                   ('score map based', 'mwsol4_resnet50_imagenet_iter_mask_cam')]:
    df = mlflow_get_runs(experiment_name, conditions)
    mapper = dict(zip(cols, cols_renamed))
    df = df.loc[:, cols].rename(columns=mapper)
    df = df.replace({'method': labels_methods})
    df = df.loc[df.method != 'Score-CAM'] # not available for score map based i.l.
    for metric in ['precision', 'recall', 'f1']:
        df[metric] *= 100 # to percentage
    df.loc[:, 'type'] = mask_type
    df_list.append(df)

# concatenate dataframes
df = pd.concat(df_list, axis=0, ignore_index=True)
df.loc[:, 'artifact_uri'] = df['artifact_uri'].str.removeprefix('file://')
df = df.sort_values(['method', 'mask'])

# plots
sns.set_style("darkgrid")
sns.set_context("paper")
markers = {'drop': 'v', 'add': '^', 'unify':'o'}
sizes = {'0.25': 50, '0.5': 100, '1.0': 150}
size_order = ['0.25', '0.5', '1.0']
hue_order = ['mean','random','zero']
sns_fig = sns.relplot(data=df, x="recall", y="precision", hue="mask", style="merge",
                      size="stop", size_order=size_order, col='method', row='type',
                      markers=markers, sizes=sizes, hue_order=hue_order, alpha=0.5)
sns.move_legend(sns_fig, loc='lower center', bbox_to_anchor=(0.6, 1.0), labelspacing=1.0, frameon=True, ncol=12)
# flatten axes into a 1-d array
# axes = sns_fig.axes.flatten()
# iterate through the axes
row_num, col_num = sns_fig.axes.shape
methods = ['cam', 'gradcam++', 'minmaxcam', 'scorecam']
for r in range(row_num):
    for c in range(col_num):
        ax = sns_fig.axes[r,c]
        method = methods[c]
        precision = prec_base['precision'][method] * 100
        recall = prec_base['recall'][method] * 100
        line_prec = ax.axhline(precision, ls='--', c='purple', label='precision', alpha=0.5)
        line_rec = ax.axvline(recall, ls='--', c='teal', label='recall', alpha=0.5)
        if r == 0 and c == 0:
            lines = [line_prec, line_rec]
            labels = ['baseline precision', 'baseline recall']
            legend = ax.legend(lines, labels, bbox_to_anchor=(0.05, 1.075), loc='lower left', title=None, frameon=True, ncols=2)
sns_fig.savefig(f'fig_iter_mask_bbox_vs_cam_resnet50_imagenet.png')

### Visual explanation

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
import pandas as pd

image_no_list = list(range(1,21))
# image_no_list = [9]#1, 2,3,4,8]

df2 = df
df2 = df2.loc[df2['method'] == 'CAM']
df2 = df2.loc[df2['merge'] == 'add']
df2 = df2.loc[df2['mask'] == 'mean']
df2 = df2.loc[df2['stop'] == '1.0']
df2 = df2.loc[:, ['method', 'type', 'artifact_uri']]
df2 = df2.sort_values(['method', 'type'])
df_dict = df2.set_index('type').to_dict()
uris = df_dict['artifact_uri']
iterations = 3
for image_no in image_no_list:
    filebase = f'ILSVRC2012_{split}_{image_no:0>8}'
    image_id = f'{split}/ILSVRC2012_{split}_{image_no:0>8}.JPEG'
    path_img_orig = os.path.join(f'/Users/goemaereg/github.com/thesis/data/dataset/ILSVRC/{image_id}')
    for r, (mask_type, uri) in enumerate(uris.items()):
        _, axes = plt.subplots(1, 1 + iterations, figsize=(16, 8))
        img = read_image(path=path_img_orig)
        axes[0].imshow(to_pil_image(img))
        axes[0].axis('off')
        class_id = class_labels[image_id]
        label_name = label_names[str(class_id)]
        axes[0].set_title(f'image ({label_name})')
        for iter in range(iterations):
            c = 1 + iter
            file_ann = f'{filebase}_ann_{iter}.png'
            path_img = os.path.join(uri, 'xai', split, file_ann)
            img = read_image(path=path_img)
            axes[c].imshow(to_pil_image(img))
            axes[c].axis('off')
            title = f'{mask_type} | iter {iter + 1}'
            axes[c].set_title(title)
        plt.show()

In [ ]:
uris